In [1]:
import json
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from urllib.request import urlopen
from googletrans import Translator, constants
from pprint import pprint

In [7]:
# df = pd.read_json('../data/taz.view_pp_ogd.json')
# df
# with open('../../data/taz.view_pp_ogd.json') as f:
#     geo_data = json.load(f)
translator = Translator()
geo_url = 'https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Oeffentlich_zugaengliche_Strassenparkplaetze_OGD?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=view_pp_ogd'

with urlopen(geo_url) as response:
    geo_data = json.load(response)

to_trans = {'Blaue Zone': 'Blue Zone',
            'Weiss markiert': 'White Zone',
            'Nur mit Geh-Behindertenausweis': 'Disabled',
            'Nur für Taxi': 'Only Taxi',
            'Für Reisecars': 'Only Coaches',
            'Für Elektrofahrzeuge': 'Only Electric vehicles',
            'Zeitweise Taxi, zeitweise Güterumschlag': 'Temporary'}

df_park = pd.json_normalize(geo_data['features'])

rename_dict = { 'properties.id1' : 'property_id', 
                'properties.parkdauer' : 'parking_duration', 
                'properties.art' : 'parking_kind',
                'properties.gebuehrenpflichtig': 'payed', 
                'properties.objectid' : 'object_id',
                'geometry.coordinates': 'coord',
}

df_park = df_park.rename(columns=rename_dict)

df_park[['lon', 'lat']] = pd.DataFrame(df_park.coord.to_list())
df_park = df_park.drop(['type', 'geometry.type', 'coord', 'id', 'object_id'], axis=1)
df_park.loc[df_park['payed'] == 'nicht gebührenpflichtig', 'payed'] = 0
df_park.loc[df_park['payed'] == 'gebührenpflichtig', 'payed'] = 1
for key in to_trans:
    df_park.loc[df_park['parking_kind'] == key, 'parking_kind'] = to_trans[key]
df_park

,parking_kind,payed,property_id,parking_duration,lon,lat
0,Blue Zone,0,206658.0,60.0,8.515525,47.328766
1,Blue Zone,0,206659.0,60.0,8.515513,47.328720
2,Blue Zone,0,206660.0,60.0,8.515630,47.329174
3,Blue Zone,0,206661.0,60.0,8.545213,47.400866
4,Blue Zone,0,206662.0,60.0,8.560581,47.406678
...,...,...,...,...,...,...
46277,Blue Zone,0,248334.0,60.0,8.513904,47.366176
46278,Blue Zone,0,248335.0,60.0,8.502234,47.374559
46279,Blue Zone,0,248336.0,60.0,8.498795,47.381467
46280,White Zone,1,248337.0,180.0,8.557656,47.372423


In [8]:
map_fig = px.scatter_mapbox(df_park, lat='lat', lon='lon', color='parking_kind')
map_fig.update_layout(mapbox_style="carto-positron",
                      mapbox_zoom=10, 
                      mapbox_center = {"lat": 47.373878, "lon": 8.545094}
                      )
map_fig.show()